<a href="https://colab.research.google.com/github/HernanJMaurice/androidSensors/blob/main/HPC/Ejercicio_1_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 22.4MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621010 sha256=294983611a305e1ce573083addd447d9a9472d08e482d7d4614934c092d3980a
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=082b0b0290e8dafcbc02767f73c268260c6a81faceec89298c6d44534236c3e0
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [8]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cadena_ADN = 'ACTG'#@param {type: "string"}

# Lo paso siempre a Mayuscula.
cadena_ADN = cadena_ADN.upper()

import re
import numpy

pattern = re.compile("(A|C|T|G)+")

# Chequeo que solo tenga Nucleotidos permitidos.
if pattern.fullmatch(cadena_ADN) is None :

    print("Los nucleotidos pueden ser: A, C, T o G.")
    quit()  

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


#Creo Array de CHAR
cadena_ADN_char = numpy.array(list(cadena_ADN))
cantidad_Nucleotidos = len(cadena_ADN_char)

cadena_ADN_Complementaria = numpy.empty_like(cadena_ADN_char)
cadena_ADN_Complementaria_Inversa  = numpy.empty_like(cadena_ADN_char)

# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


# CPU - reservo la memoria GPU.
cadena_ADN_char_GPU = cuda.mem_alloc( cadena_ADN_Complementaria.nbytes )
cadena_ADN_Complementaria_GPU = cuda.mem_alloc( cadena_ADN_Complementaria.nbytes )
cadena_ADN_Complementaria_Inversa_GPU = cuda.mem_alloc( cadena_ADN_Complementaria_Inversa.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( cadena_ADN_char_GPU, cadena_ADN_char )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_secADN( int n, char *origen, char *complementario, char *complementario_inversa )
{
  int indice = threadIdx.x + blockIdx.x*blockDim.x;
  if( indice < n )
  {
     if (origen[indice] =='A')
     {
        complementario[indice] = 'T';
        complementario_inversa[n-indice-1] ='T';
     }  
    else if(origen[indice] =='C')
    {
       complementario[indice] = 'G';
        complementario_inversa[n-indice-1] ='G';
    }else if( origen[indice] =='G')
    {
        complementario[indice] = 'C';
        complementario_inversa[n-indice-1] ='C';
    }else if(origen[indice] =='T' )
    {
        complementario[indice] = 'A' ;
        complementario_inversa[n-indice-1] ='A';
    }
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_secADN")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_Nucleotidos+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_Nucleotidos), cadena_ADN_char_GPU, cadena_ADN_Complementaria_GPU, cadena_ADN_Complementaria_Inversa_GPU, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( cadena_ADN_Complementaria, cadena_ADN_Complementaria_GPU )
cuda.memcpy_dtoh( cadena_ADN_Complementaria_Inversa, cadena_ADN_Complementaria_Inversa_GPU )

"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )
"""

tiempo_total = datetime.now() - tiempo_total


print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Cadena ADN original: ", cadena_ADN_char )
print("Cadena ADN Complementaria: ", cadena_ADN_Complementaria )
print("Cadena ADN Complementaria Inversa: ",  cadena_ADN_Complementaria_Inversa )
print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo bucle: ", tiempo_en_ms( tiempo_gpu ), "[ms]" )

Thread x:  256 , Bloque x: 1
Thread x:  256 , Bloque x: 1
Cadena ADN original:  ['A' 'C' 'T' 'G']
Cadena ADN Complementaria:  ['T' 'C' 'T' 'G']
Cadena ADN Complementaria Inversa:  

UnicodeDecodeError: ignored